In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
train = pd.read_csv('input/train.csv')
client = pd.read_csv('input/client.csv')
electricity_prices = pd.read_csv('input/electricity_prices.csv')
forecast_weather = pd.read_csv('input/forecast_weather.csv')
gas_prices = pd.read_csv('input/gas_prices.csv')
historical_weather = pd.read_csv('input/historical_weather.csv')
train = train.dropna()

In [4]:
train['datetime'] = pd.to_datetime(train.datetime)
earliest = train['datetime'].min()
train['day_no'] = (train['datetime'] - earliest).dt.days
train['hour'] = train['datetime'].apply(lambda x: x.hour)
train['month'] = train['datetime'].apply(lambda x: x.month)
train['weekday'] = train['datetime'].apply(lambda x: x.dayofweek)
train.head(3)

,county,is_business,product_type,target,is_consumption,datetime,data_block_id,row_id,prediction_unit_id,day_no,hour,month,weekday
0,0,0,1,0.713,0,2021-09-01,0,0,0,0,0,9,2
1,0,0,1,96.590,1,2021-09-01,0,1,0,0,0,9,2
2,0,0,2,0.000,0,2021-09-01,0,2,1,0,0,9,2


In [5]:
train2 = train.copy()
train2['product_type'] = (train2['product_type'] + 1) % 4
X = train2[['is_business', 'is_consumption', 'product_type', 'day_no', 'hour', 'month', 'weekday']]
y = train2['target']

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

X_train_and_validate, X_test, y_train_and_validate, y_test = train_test_split(X, y, test_size=200000, random_state=0)
X_train, X_valid, y_train, y_valid =  train_test_split(X_train_and_validate, y_train_and_validate, test_size=200000, random_state=0)

In [7]:
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_predict_train = linreg.predict(X_train)
y_predict_valid = linreg.predict(X_valid)
print('MAE: training set')
print(mean_absolute_error(y_predict_train, y_train))
print('MAE: validation set')
print(mean_absolute_error(y_predict_valid, y_valid))

MAE: training set
361.855556550744
MAE: validation set
359.9827299547037


In [26]:
county_map = {
    (24.7, 59.4): 0,
    (22.7, 58.8): 1,
    (27.2, 59.4): 2,
    (25.7, 58.8): 3,
    (26.2, 58.8): 4,
    (26.2, 59.4): 5,
    (23.7, 58.8): 6,
    (24.2, 58.5): 7,
    (27.2, 58.2): 8,
    (24.7, 58.8): 9,
    (22.7, 58.5): 10,
    (26.7, 58.5): 11,
    (24.7, 59.1): 12,
    (26.2, 57.9): 13,
    (25.2, 58.5): 14
}
def get_county_id(row):
    lat, lon = row['latitude'], row['longitude']
    if int(row.name) % 100000 == 0:
        print(row.name, '/', len(forecast_weather))
    if (lon, lat) in county_map:
        row['county_id'] = county_map[(lon, lat)]
    return row
forecast_weather2 = forecast_weather.apply(get_county_id, axis=1)
forecast_weather2.isnull().sum()

0 / 3424512
100000 / 3424512
200000 / 3424512
300000 / 3424512
400000 / 3424512
500000 / 3424512
600000 / 3424512
700000 / 3424512
800000 / 3424512
900000 / 3424512
1000000 / 3424512
1100000 / 3424512
1200000 / 3424512
1300000 / 3424512
1400000 / 3424512
1500000 / 3424512
1600000 / 3424512
1700000 / 3424512
1800000 / 3424512
1900000 / 3424512
2000000 / 3424512
2100000 / 3424512
2200000 / 3424512
2300000 / 3424512
2400000 / 3424512
2500000 / 3424512
2600000 / 3424512
2700000 / 3424512
2800000 / 3424512
2900000 / 3424512
3000000 / 3424512
3100000 / 3424512
3200000 / 3424512
3300000 / 3424512
3400000 / 3424512


10_metre_u_wind_component                  0
10_metre_v_wind_component                  0
cloudcover_high                            0
cloudcover_low                             0
cloudcover_mid                             0
cloudcover_total                           0
county_id                            2965872
data_block_id                              0
dewpoint                                   0
direct_solar_radiation                     0
forecast_datetime                          0
hours_ahead                                0
latitude                                   0
longitude                                  0
origin_datetime                            0
snowfall                                   0
surface_solar_radiation_downwards          2
temperature                                0
total_precipitation                        0
dtype: int64

In [8]:
# forecast_weather3 = forecast_weather2.dropna()
# forecast_weather3.to_csv('output/forecast_weather_county_id.csv')
# forecast_weather3.head(3)
forecast_weather3 = pd.read_csv('output/forecast_weather_county_id.csv')
forecast_weather3.head()

,Unnamed: 0,10_metre_u_wind_component,10_metre_v_wind_component,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,county_id,data_block_id,dewpoint,direct_solar_radiation,forecast_datetime,hours_ahead,latitude,longitude,origin_datetime,snowfall,surface_solar_radiation_downwards,temperature,total_precipitation
0,23,1.523974,-2.439878,0.248322,0.104279,0.066223,0.332809,13.0,1,9.031396,0.0,2021-09-01 01:00:00+00:00,1,57.9,26.2,2021-09-01 00:00:00+00:00,0.0,0.0,10.058984,0.0
1,39,2.072436,-1.438657,0.000000,0.076691,0.133987,0.143875,8.0,1,8.484277,0.0,2021-09-01 01:00:00+00:00,1,58.2,27.2,2021-09-01 00:00:00+00:00,0.0,0.0,9.637109,0.0
2,44,1.645679,-4.721860,0.462769,0.030365,0.000000,0.479096,10.0,1,10.484766,0.0,2021-09-01 01:00:00+00:00,1,58.5,22.7,2021-09-01 00:00:00+00:00,0.0,0.0,12.734277,0.0
3,47,1.527026,-3.641782,0.001282,0.425385,0.000290,0.426010,7.0,1,9.743921,0.0,2021-09-01 01:00:00+00:00,1,58.5,24.2,2021-09-01 00:00:00+00:00,0.0,0.0,12.436060,0.0
4,49,1.141894,-3.007016,0.179169,0.936737,0.496582,0.952545,14.0,1,10.502100,0.0,2021-09-01 01:00:00+00:00,1,58.5,25.2,2021-09-01 00:00:00+00:00,0.0,0.0,12.386133,0.0


In [9]:
earliest = train['datetime'].min()
forecast_weather3['origin_datetime'] = pd.to_datetime(forecast_weather3['origin_datetime'], utc=True)
forecast_weather3['day_no'] = (forecast_weather3['origin_datetime'] - earliest.tz_localize('UTC')).dt.days

In [15]:
train3 = train2[train2['day_no'] < 100]
forecast_weather4 = forecast_weather3[forecast_weather3['day_no'] < 100]

train4 = pd.merge(train3[1:2], forecast_weather4, how='left', left_on=['county','day_no'], right_on = ['county_id','day_no'])
train4

,county,is_business,product_type,target,is_consumption,datetime,data_block_id_x,row_id,prediction_unit_id,day_no,...,direct_solar_radiation,forecast_datetime,hours_ahead,latitude,longitude,origin_datetime,snowfall,surface_solar_radiation_downwards,temperature,total_precipitation
0,0,0,2,96.59,1,2021-09-01,0,1,0,0,...,0.000000,2021-09-01 01:00:00+00:00,1,59.4,24.7,2021-09-01 00:00:00+00:00,0.0,0.000000,12.374780,0.000000e+00
1,0,0,2,96.59,1,2021-09-01,0,1,0,0,...,0.000000,2021-09-01 02:00:00+00:00,2,59.4,24.7,2021-09-01 00:00:00+00:00,0.0,0.000000,11.488306,0.000000e+00
2,0,0,2,96.59,1,2021-09-01,0,1,0,0,...,0.000000,2021-09-01 03:00:00+00:00,3,59.4,24.7,2021-09-01 00:00:00+00:00,0.0,0.000000,11.484277,0.000000e+00
3,0,0,2,96.59,1,2021-09-01,0,1,0,0,...,56.053333,2021-09-01 04:00:00+00:00,4,59.4,24.7,2021-09-01 00:00:00+00:00,0.0,9.321972,11.999292,7.152557e-07
4,0,0,2,96.59,1,2021-09-01,0,1,0,0,...,210.811163,2021-09-01 05:00:00+00:00,5,59.4,24.7,2021-09-01 00:00:00+00:00,0.0,69.088653,12.911523,2.384186e-06
5,0,0,2,96.59,1,2021-09-01,0,1,0,0,...,404.164340,2021-09-01 06:00:00+00:00,6,59.4,24.7,2021-09-01 00:00:00+00:00,0.0,178.996302,13.520288,0.000000e+00
6,0,0,2,96.59,1,2021-09-01,0,1,0,0,...,607.579410,2021-09-01 07:00:00+00:00,7,59.4,24.7,2021-09-01 00:00:00+00:00,0.0,321.074045,13.622705,-2.384186e-06
7,0,0,2,96.59,1,2021-09-01,0,1,0,0,...,763.350503,2021-09-01 08:00:00+00:00,8,59.4,24.7,2021-09-01 00:00:00+00:00,0.0,453.948194,14.033044,0.000000e+00
8,0,0,2,96.59,1,2021-09-01,0,1,0,0,...,824.603194,2021-09-01 09:00:00+00:00,9,59.4,24.7,2021-09-01 00:00:00+00:00,0.0,550.344444,14.347803,0.000000e+00
9,0,0,2,96.59,1,2021-09-01,0,1,0,0,...,825.445000,2021-09-01 10:00:00+00:00,10,59.4,24.7,2021-09-01 00:00:00+00:00,0.0,595.539722,14.740625,0.000000e+00
